In [ ]:
#run this block if the pyspark library is not installed
!pip install pyspark

     |████████████████████████████████| 281.4 MB 31 kB/s 
     |████████████████████████████████| 198 kB 48.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=c768608b32bc1d1903a8d28efda97fa8215b846a14ba5a597826e7c9fe142025
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
# installation required to run the session
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import pyspark.sql.functions as func
import time

In [ ]:
#The entry point into all functionality in Spark is the SparkSession class. 
#To create a basic SparkSession, just use SparkSession.builder()

# Initialize SparkSession
spark = SparkSession \
    .builder \
    .appName("Group_25") \
    .getOrCreate()


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving nation_DT.csv to nation_DT.csv


For each nation shows the GDP and number of medals, ordered by GDP

In [ ]:
datasetPath = 'nation_DT.csv'

In [ ]:
# Summary of schema and the data in the CSV file of BigMac Burgers Dataset
datasetDF = spark.read.csv(datasetPath, header=True, inferSchema=True)
datasetDF.printSchema()

root
 |-- ID_nation: integer (nullable = true)
 |-- Nation: string (nullable = true)
 |-- Population: integer (nullable = true)
 |-- GDP per Capita: integer (nullable = true)
 |-- Code: string (nullable = true)
 |-- Nbr_apps: integer (nullable = true)
 |-- Nbr_medal : integer (nullable = true)
 |-- Nbr_gold: integer (nullable = true)
 |-- Nbr_silver: integer (nullable = true)
 |-- Nbr_bronze: integer (nullable = true)
 |-- Nbr_SO_medal: integer (nullable = true)
 |-- Nbr_WO_medal: integer (nullable = true)
 |-- Most_succ_sport: string (nullable = true)



In [ ]:

data = datasetDF.select('Nation', 'GDP per Capita', 'Nbr_medal ').filter(col('GDP per Capita').isNotNull()).sort(desc('GDP per Capita')).show()

+--------------------+--------------+----------+
|              Nation|GDP per Capita|Nbr_medal |
+--------------------+--------------+----------+
|          Uzbekistan|    2132070368|        32|
|             Ukraine|    2114954716|       129|
|             Vietnam|    2111138024|         4|
|           Nicaragua|    2086895003|         0|
|            Slovenia|    2072653989|        40|
|        Saudi Arabia|    2048174532|         3|
|            Djibouti|    1945124293|         1|
|     Solomon Islands|    1934856293|         0|
|            Portugal|    1922215008|        24|
|             Ecuador|    1910096849|         2|
|            Ethiopia|    1896726769|        54|
|               Congo|    1851199916|         0|
|             Moldova|    1848061804|         5|
|                Laos|    1818441373|         0|
|             Ireland|    1818399947|        31|
|              Greece|    1800223058|       116|
|            Colombia|    1761180421|        22|
|      Czech Republi

Count the number of medals of the female athletes that belong to Iraq, Iran and Afghanistan

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving olympic_athletes (1).csv to olympic_athletes (1) (1).csv
Saving olympic_event_FT.csv to olympic_event_FT.csv


In [ ]:
event_datasetPath = 'olympic_event_FT.csv'
athletes_datasetPath = 'olympic_athletes (1).csv'

In [ ]:
# Summary of schema and the data in the CSV file of BigMac Burgers Dataset
event_datasetDF = spark.read.csv(event_datasetPath, header=True, inferSchema=True)
event_datasetDF.printSchema()

root
 |-- Id_olympic_event: integer (nullable = true)
 |-- Games: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Medal: string (nullable = true)
 |-- ID_athlete: integer (nullable = true)
 |-- ID_event: integer (nullable = true)
 |-- ID_host_city: integer (nullable = true)



In [ ]:
athletes_datasetDF = spark.read.csv(athletes_datasetPath, header=True, inferSchema=True)
athletes_datasetDF.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- Code: string (nullable = true)
 |-- ID_athlete: string (nullable = true)
 |-- ID_nation: string (nullable = true)



In [ ]:
athletes_datasetDF.join(event_datasetDF,athletes_datasetDF.ID_athlete ==  event_datasetDF.ID_athlete,"inner") \
     .where(((col('Team')=='Iraq') | (col('Team')=='Iran') | (col('Team')=='Afghanistan')) & (col('Sex')=='F')).count()

61

Select for each year the number of medals of the USA

In [ ]:
athletes_datasetDF.join(event_datasetDF,athletes_datasetDF.ID_athlete ==  event_datasetDF.ID_athlete,"inner") \
     .where((col('Code') == 'USA') & (col('Medal') != 'NA')).selectExpr('Code', 'Year','Medal').groupBy('Code', 'Year').agg(F.count('Medal').alias('nbr_medals')).sort('year').show()

+----+----+----------+
|Code|Year|nbr_medals|
+----+----+----------+
| USA|1896|        20|
| USA|1900|        62|
| USA|1904|       390|
| USA|1906|        24|
| USA|1908|        63|
| USA|1912|       105|
| USA|1920|       187|
| USA|1924|       187|
| USA|1928|        97|
| USA|1932|       208|
| USA|1936|       105|
| USA|1948|       155|
| USA|1952|       151|
| USA|1956|       137|
| USA|1960|       141|
| USA|1964|       161|
| USA|1968|       163|
| USA|1972|       188|
| USA|1976|       166|
| USA|1980|        30|
+----+----+----------+
only showing top 20 rows



For the nations that have a population greater than 80 billions show the number of medals

SELECT nation, population, nbr_medal FROM "LogicalSchema_Project_DW".nation_dt
where population >= 80000000


In [ ]:
data = datasetDF.select('Nation', 'Population', 'Nbr_medal ').filter(col('Population')>='80000000').show()

+-------------+----------+----------+
|       Nation|Population|Nbr_medal |
+-------------+----------+----------+
|   Bangladesh| 160995642|         0|
|       Brazil| 207847528|       129|
|        China|1371220000|       608|
|        Egypt|  91508084|        32|
|     Ethiopia|  99390750|        54|
|      Germany|  81413145|       855|
|        India|1311050527|        28|
|    Indonesia| 257563815|        32|
|        Japan| 126958472|       497|
|       Mexico| 127017224|        69|
|      Nigeria| 182201962|        25|
|     Pakistan| 188924874|        10|
|  Philippines| 100699395|        10|
|       Russia| 144096812|       548|
|United States| 321418820|      2827|
|      Vietnam|  91703800|         4|
+-------------+----------+----------+



For each nations that has won more than 50 medals in the summer olympics show the medals relative to the winter olympics
SELECT nation, nbr_SO_medal, nbr_WO_medal FROM
"LogicalSchema_Project_DW".nation_dt
WHERE nbr_SO_medal >= 50


In [ ]:
data = datasetDF.select('Nation', 'Nbr_SO_medal', 'Nbr_WO_medal').filter(col('nbr_SO_medal')>='50').show()

+--------------+------------+------------+
|        Nation|Nbr_SO_medal|Nbr_WO_medal|
+--------------+------------+------------+
|     Argentina|          74|           0|
|     Australia|         497|          15|
|       Austria|          87|         232|
|       Belarus|          78|          18|
|       Belgium|         148|           6|
|        Brazil|         129|           0|
|      Bulgaria|         218|           6|
|        Canada|         302|         199|
|         China|         546|          62|
|          Cuba|         226|           0|
|Czech Republic|          56|          31|
|       Denmark|         194|           1|
|      Ethiopia|          54|           0|
|       Finland|         303|         167|
|        France|         716|         124|
|       Germany|         615|         240|
|United Kingdom|         851|          32|
|        Greece|         116|           0|
|       Hungary|         491|           7|
|          Iran|          69|           0|
+----------

Olympic_participation(Nation.most_succ_sport; Athlete.code=”NOR” & Athlete.code=”SWE” & Athlete.code=”FIN” & Athlete.code=”DEN” & Athlete.code=”ISL” & Athlete.code=”IRL”)
Show the sport for which the athletes of the nordic states have won more medals

SELECT nation, most_succ_sport FROM
"LogicalSchema_Project_DW".nation_dt
WHERE code = 'NOR' OR code = 'SWE' OR code = 'FIN' OR code = 'DEN' OR code = 'ISL' OR code = 'IRL'


In [ ]:
data = datasetDF.select('Nation', 'most_succ_sport')\
.where((col('code')=='NOR')|(col('code')=='SWE')|(col('code')=='FIN')|(col('code')=='DEN')|(col('code')=='ISL')|(col('code')=='IRL')).show()

+-------+--------------------+
| Nation|     most_succ_sport|
+-------+--------------------+
|Denmark|             Sailing|
|Finland|           Athletics|
|Iceland|            Handball|
|Ireland|              Boxing|
| Norway|Cross Country Skiing|
| Sweden|Cross country skiing|
+-------+--------------------+



SELECT nation, nbr_medal FROM "LogicalSchema_Project_DW".nation_dt
order by nbr_medal desc
limit 10
Olympic_participation(Nation.Nations,Nation.nbr_medal)
Order by number of medals and select the ten best Nations and the relative number
 


In [ ]:
data = datasetDF.select('Nation', 'Nbr_medal ').sort(desc('nbr_medal ')).show(10)

+--------------+----------+
|        Nation|Nbr_medal |
+--------------+----------+
| United States|      2827|
|United Kingdom|       883|
|       Germany|       855|
|        France|       840|
|         Italy|       701|
|        Sweden|       652|
|         China|       608|
|        Russia|       548|
|        Norway|       520|
|     Australia|       512|
+--------------+----------+
only showing top 10 rows



Olympic_participation(Nation.nation, Athlete.name; Nation.nation.GDP >= 20.000 )
Select the nation that has a GDP greater than 20.000 and show the relative number of athletes


 SELECT nation, gdp, count(name) as nbr_athletes FROM 
 
 "LogicalSchema_Project_DW".olympic_events_ft 

join
(
	SELECT * FROM "LogicalSchema_Project_DW".athlete_dt join "LogicalSchema_Project_DW".nation_dt on
	"LogicalSchema_Project_DW".athlete_dt.code = "LogicalSchema_Project_DW".nation_dt.code

) as t1 on "LogicalSchema_Project_DW".olympic_events_ft.id_athlete = t1.

id_athlete
where gdp is not null and gdp >= 20000
GROUP BY (nation, gdp) order by gdp Desc  


In [ ]:
df = athletes_datasetDF.join(datasetDF,athletes_datasetDF.Code ==  datasetDF.Code,"inner")
event_datasetDF.join(df,event_datasetDF.ID_athlete == df.ID_athlete, "inner").where((col('GDP per Capita')>='20000')&(col('GDP per Capita').isNotNull()))\
.groupBy('Nation', 'GDP per Capita').agg(F.count('Name').alias('nbr_athlets')).sort(desc(col('GDP per Capita'))).show()


+--------------------+--------------+-----------+
|              Nation|GDP per Capita|nbr_athlets|
+--------------------+--------------+-----------+
|          Uzbekistan|    2132070368|        471|
|             Ukraine|    2114954716|       2537|
|             Vietnam|    2111138024|        154|
|           Nicaragua|    2086895003|        129|
|            Slovenia|    2072653989|       1112|
|        Saudi Arabia|    2048174532|        230|
|            Djibouti|    1945124293|         44|
|     Solomon Islands|    1934856293|         27|
|            Portugal|    1922215008|       1530|
|             Ecuador|    1910096849|        275|
|            Ethiopia|    1896726769|        378|
|               Congo|    1851199916|        104|
|             Moldova|    1848061804|        243|
|                Laos|    1818441373|         54|
|             Ireland|    1818399947|       1324|
|              Greece|    1800223058|       3181|
|            Colombia|    1761180421|       1069|


Olympic_participation( Event.Sport, Day.Year; Day.Year= “2016” & Day.Year=”1896”)
Count the number of each sport in the year 2016 and 1896 pick, for each of the two years, the sports with the largest number of events


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving event_DT.csv to event_DT.csv


In [ ]:
athletic_event_datasetDF = event_datasetDF
event_datasetDF = spark.read.csv('event_DT.csv', header=True, inferSchema=True)
event_datasetDF.printSchema()

root
 |-- Id_event: integer (nullable = true)
 |-- Event: string (nullable = true)
 |-- Sport: string (nullable = true)



In [ ]:
df = event_datasetDF.join(athletic_event_datasetDF,event_datasetDF.Id_event ==  athletic_event_datasetDF.ID_event,"inner")
df = df.where(col('Year')=='1896').groupBy('Sport','Year').agg(F.count('Event').alias('nbr_events'))
df = df.sort(desc('nbr_events')).limit(1)
df.show()

+---------+----+----------+
|    Sport|Year|nbr_events|
+---------+----+----------+
|Athletics|1896|       106|
+---------+----+----------+



In [ ]:
df_2 = event_datasetDF.join(athletic_event_datasetDF,event_datasetDF.Id_event ==  athletic_event_datasetDF.ID_event,"inner")
df_2 = df_2.where(col('Year')=='2016').groupBy('Sport','Year').agg(F.count('Event').alias('nbr_events'))
df_2 = df_2.sort(desc('nbr_events')).limit(1)
df_2.show()

+---------+----+----------+
|    Sport|Year|nbr_events|
+---------+----+----------+
|Athletics|2016|      2508|
+---------+----+----------+



In [ ]:
union_df = df.union(df_2).show()

+---------+----+----------+
|    Sport|Year|nbr_events|
+---------+----+----------+
|Athletics|1896|       106|
|Athletics|2016|      2508|
+---------+----+----------+

